In [1]:
import random
import numpy as np
import array

from representations import INSERTABLE, MUTABLE_PARAMS, default_init_nn_repr, REPR_MAKERS
import numpy as np
import random

from deap import algorithms
from deap import base
from deap import creator
from deap import tools

import representations, fitness, mutations


#INITIAL_BLOCKS = 5 # Represents how many random layer blocks each NNet should start with
INITIAL_BLOCKS = 5 # Random Exclusive represents how many random layer blocks each NNet should start with

POPULATION = 10
GENERATIONS = 10
PROB_MUTATIONS = 0.25 # Probability of mutating in a new generation
PROB_MATE = 0.6 # Probability of mating / crossover in a new generation
NUMBER_EPOCHS = 2 #Epochs when training the network

#---------------------

def getRandomIndividual(iterations=1):
    #Possible networks to choose from:
    '''
    networks = [
        [representations.make_conv2d_repr(),
        representations.make_pool_repr()],

        [representations.make_dropout_repr(),
        representations.make_conv2d_repr()],

        [representations.make_batchnorm_repr()],

        [representations.make_noise_repr()]
    ]

    probabilities = [0.3, 0.3, 0.25, 0.15]

    
    out = []

    
    for x in range(0,iterations):
        choice = np.random.choice(networks, p=probabilities)
        for layer in choice:
            out.append(layer)
    
    return out
    '''

    # get this list from representations.REPR_MAKERS. If you add a new block type,
    # also add it to the representations.REPR_MAKERS list, so it can also mutate.
    networks = [
        representations.make_conv2d_pool_repr(),
        representations.make_conv2d_dropout_repr(),
        representations.make_batchnorm_repr(),
        representations.make_noise_repr(),
        representations.make_dropout_repr(),

    ]
    probabilities = [0.3, 0.3, 0.10, 0.15, 0.15]
    
    return np.random.choice(networks,p=probabilities)


'''
Evaluation function (should return the fitness)
'''
def evaluateFunc(individual):
    return fitness.evaluate_nn(individual, NUMBER_EPOCHS), #<--- IMPORTANT: add the comma ','; as it needs to return a tuple

def initRepeatRandom(container, func, n):
    """
    Extended toolbox.initRepeat() function to work with random initialization instead of fixed numbers.
    """
    return container(func() for _ in range(np.random.randint(1,n)))

# -------------- Init / Main stuff ----------------------

# Create attributes
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", np.ndarray,  fitness=creator.FitnessMax)

mutations.setIndividual(creator.Individual)


toolbox = base.Toolbox()

toolbox.register("individual", initRepeatRandom, creator.Individual, getRandomIndividual, n=INITIAL_BLOCKS) #<-- Creates 3 elements. This random, however does not evaluate the random function each time yet
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


toolbox.register("evaluate", evaluateFunc) #register the evaluation function
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", mutations.mutate_layer, verbose=True)
toolbox.register("select", tools.selTournament, tournsize=3)
#toolbox.register("select", tools.selBest)
#deap.tools.selBest(individuals, k, fit_attr='fitness')¶



c:\users\ruben\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
random.seed(1337)
pop = toolbox.population(n=POPULATION)



hof = tools.HallOfFame(10, similar=np.array_equal)

stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

In [3]:
containerIndividual = mutations.containerIndividual

In [4]:
indiv = pop[1]

In [5]:
indiv

Individual([{'type': 'dropout', 'params': {'rate': 0.37}},
            {'type': 'noise', 'params': {'stddev': 0.3509362915721842}},
            {'type': 'conv2dpool', 'params': {'filters': 32, 'kernel_size': 3, 'activation': 'relu', 'pool_size': 1}}],
           dtype=object)

In [6]:
newArr = [indiv[1], indiv[2]] # append  just 2 of 3 layers

In [7]:
newArr

[{'params': {'stddev': 0.3509362915721842}, 'type': 'noise'},
 {'params': {'activation': 'relu',
   'filters': 32,
   'kernel_size': 3,
   'pool_size': 1},
  'type': 'conv2dpool'}]

In [8]:
containerIndividual(newArr)

Individual([{'type': 'noise', 'params': {'stddev': 0.3509362915721842}},
            {'type': 'conv2dpool', 'params': {'filters': 32, 'kernel_size': 3, 'activation': 'relu', 'pool_size': 1}}],
           dtype=object)

In [20]:
def mutate_layer(layer, verbose=False):
    '''
    Looks up the initializer function for a type,
    and replaces it with a new initialization.

    Appended [0] because you need to access the container inside the
    Individual({type: conv2dpool, params})
    class.    
    '''
    for elem in REPR_MAKERS:
        if layer['type'] == elem:
            layer = REPR_MAKERS[elem]()
            if verbose:
                print('MUTATED LAYER %s' % layer['type'])
    return layer

def mutate_network(reprRaw, mutations=2, verbose=False):
    '''
    @Deprecated
    Mutates a whole representation network.
    Apparently, the mutate function of deap does select a layer of itself to mutate, not a whole network.
    '''
    repr = reprRaw.tolist()

    if mutations > len(repr):
        mutations = len(repr)-1 # prevents setting higher count of mutations than length of representation

    if verbose:
        print("MUTATING %d BLOCKS OF NETWORK" % mutations)
    
    for layerIndex in np.random.randint(0, len(repr), mutations):
        if verbose:
            repr[layerIndex] = mutate_layer(repr[layerIndex], verbose=True)
        else:
            repr[layerIndex] = mutate_layer(repr[layerIndex])
    return containerIndividual(repr)

In [22]:
mutate_network(indiv, verbose=True)

MUTATING 2 BLOCKS OF NETWORK
MUTATED LAYER noise
MUTATED LAYER conv2dpool


Individual([{'type': 'dropout', 'params': {'rate': 0.37}},
            {'type': 'noise', 'params': {'stddev': 0.7265262941156184}},
            {'type': 'conv2dpool', 'params': {'filters': 32, 'kernel_size': 3, 'activation': 'relu', 'pool_size': 1}}],
           dtype=object)

In [23]:
indiv

Individual([{'type': 'dropout', 'params': {'rate': 0.37}},
            {'type': 'noise', 'params': {'stddev': 0.3509362915721842}},
            {'type': 'conv2dpool', 'params': {'filters': 32, 'kernel_size': 3, 'activation': 'relu', 'pool_size': 1}}],
           dtype=object)

In [37]:
list = indiv.tolist()
list

[{'params': {'rate': 0.37}, 'type': 'dropout'},
 {'params': {'stddev': 0.3509362915721842}, 'type': 'noise'},
 {'params': {'activation': 'relu',
   'filters': 32,
   'kernel_size': 3,
   'pool_size': 1},
  'type': 'conv2dpool'}]

In [50]:
list2 = random.shuffle(list)

In [51]:
list

[{'params': {'activation': 'relu',
   'filters': 32,
   'kernel_size': 3,
   'pool_size': 1},
  'type': 'conv2dpool'},
 {'params': {'rate': 0.37}, 'type': 'dropout'},
 {'params': {'stddev': 0.3509362915721842}, 'type': 'noise'}]